<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Pascal-in-Pandas" data-toc-modified-id="Pascal-in-Pandas-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pascal in Pandas</a></span></li></ul></div>

# Pascal in Pandas

[Based on this Gist](https://gist.github.com/binga/1bc4ebe5e41f670f5954d2ffa9d6c0ed)

In [11]:
import json
import pandas as pd
from pathlib import Path

models/                pascal_train2012.json  tmp/
pascal_test2007.json   pascal_val2007.json    VOC2007_doc.pdf
pascal_train2007.json  pascal_val2012.json    VOCdevkit/


In [12]:
PATH = Path('data/pascal')
FN = str(PATH)+"/pascal_train2007.json"

In [13]:
with open(FN) as f:
    d = json.load(f)
    
print(d.keys())
    
categories = pd.DataFrame(d['categories'])
annotations = pd.DataFrame(d['annotations'])
images = pd.DataFrame(d['images'])

dict_keys(['images', 'type', 'annotations', 'categories'])


In [14]:
images.head()

,file_name,height,id,width
0,000012.jpg,333,12,500
1,000017.jpg,364,17,480
2,000023.jpg,500,23,334
3,000026.jpg,333,26,500
4,000032.jpg,281,32,500


In [15]:
categories.head()

,id,name,supercategory
0,1,aeroplane,none
1,2,bicycle,none
2,3,bird,none
3,4,boat,none
4,5,bottle,none


In [16]:
annotations.head()

,area,bbox,category_id,id,ignore,image_id,iscrowd,segmentation
0,34104,"[155, 96, 196, 174]",7,1,0,12,0,"[[155, 96, 155, 270, 351, 270, 351, 96]]"
1,13110,"[184, 61, 95, 138]",15,2,0,17,0,"[[184, 61, 184, 199, 279, 199, 279, 61]]"
2,81326,"[89, 77, 314, 259]",13,3,0,17,0,"[[89, 77, 89, 336, 403, 336, 403, 77]]"
3,64227,"[8, 229, 237, 271]",2,4,0,23,0,"[[8, 229, 8, 500, 245, 500, 245, 229]]"
4,29505,"[229, 219, 105, 281]",2,5,0,23,0,"[[229, 219, 229, 500, 334, 500, 334, 219]]"


In [17]:
data = (annotations
        .merge(categories, how='left', left_on='category_id', right_on='id')
        .merge(images, how='left', left_on='image_id', right_on='id'))
data.head()

,area,bbox,category_id,id_x,ignore,image_id,iscrowd,segmentation,id_y,name,supercategory,file_name,height,id,width
0,34104,"[155, 96, 196, 174]",7,1,0,12,0,"[[155, 96, 155, 270, 351, 270, 351, 96]]",7,car,none,000012.jpg,333,12,500
1,13110,"[184, 61, 95, 138]",15,2,0,17,0,"[[184, 61, 184, 199, 279, 199, 279, 61]]",15,person,none,000017.jpg,364,17,480
2,81326,"[89, 77, 314, 259]",13,3,0,17,0,"[[89, 77, 89, 336, 403, 336, 403, 77]]",13,horse,none,000017.jpg,364,17,480
3,64227,"[8, 229, 237, 271]",2,4,0,23,0,"[[8, 229, 8, 500, 245, 500, 245, 229]]",2,bicycle,none,000023.jpg,500,23,334
4,29505,"[229, 219, 105, 281]",2,5,0,23,0,"[[229, 219, 229, 500, 334, 500, 334, 219]]",2,bicycle,none,000023.jpg,500,23,334


In [18]:
largest_bbox = data.pivot_table(index='file_name', values='area', aggfunc=max).reset_index()
largest_bbox = largest_bbox.merge(data[['area', 'bbox', 'image_id', 'file_name', 'name']], how='left')
largest_bbox.head()

,file_name,area,bbox,image_id,name
0,000012.jpg,34104,"[155, 96, 196, 174]",12,car
1,000017.jpg,81326,"[89, 77, 314, 259]",17,horse
2,000023.jpg,111101,"[2, 1, 241, 461]",23,person
3,000026.jpg,21824,"[89, 124, 248, 88]",26,car
4,000032.jpg,28832,"[103, 77, 272, 106]",32,aeroplane


In [19]:
def bb_hw_pandas(x):
    return [x[1], x[0], x[1]+x[3]-1, x[0]+x[2]-1]

largest_bbox['bbox_new'] = largest_bbox['bbox'].apply(lambda x: bb_hw_pandas(x))
largest_bbox['bbox_str'] = largest_bbox['bbox_new'].apply(lambda x: ' '.join(str(y) for y in x))
largest_bbox.head()

,file_name,area,bbox,image_id,name,bbox_new,bbox_str
0,000012.jpg,34104,"[155, 96, 196, 174]",12,car,"[96, 155, 269, 350]",96 155 269 350
1,000017.jpg,81326,"[89, 77, 314, 259]",17,horse,"[77, 89, 335, 402]",77 89 335 402
2,000023.jpg,111101,"[2, 1, 241, 461]",23,person,"[1, 2, 461, 242]",1 2 461 242
3,000026.jpg,21824,"[89, 124, 248, 88]",26,car,"[124, 89, 211, 336]",124 89 211 336
4,000032.jpg,28832,"[103, 77, 272, 106]",32,aeroplane,"[77, 103, 182, 374]",77 103 182 374


In [20]:
f = "bbox_dataset.csv"
largest_bbox[['file_name', 'bbox_str']].to_csv(f, index=False)

In [21]:
largest_bbox[['file_name', 'bbox_str']].head()

,file_name,bbox_str
0,000012.jpg,96 155 269 350
1,000017.jpg,77 89 335 402
2,000023.jpg,1 2 461 242
3,000026.jpg,124 89 211 336
4,000032.jpg,77 103 182 374
